In [8]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
file_path = r"C:\Users\lenovo‘\Desktop\projectdata.xlsx"
df= pd.read_excel(file_path, engine='openpyxl')
df.head()

,date,price,logreturn,HS300,CompriceIndex,PE,S&P500Volatility3,deprate,deprate3,deprate6,...,setpriceCOMEXsilver,Londonspotgold,Londonspotsilver,SHgoldspotvolumeAu9999,SHgoldspotClspriceAu9999,Goldproduction,Goldcondemand,USGoldcondemand,tradevolume,position
0,2012-01-05,331.95,0.014110,2276.385,0.00,13.9824,24.70,0.5,3.1,3.3,...,29.296,1032.546,28.92,7869.6,331.98,19.733,259.79,14.05,82316,99406
1,2012-01-06,333.41,0.004389,2290.601,155.28,13.9454,24.09,0.5,3.1,3.3,...,28.683,1049.880,29.40,9875.8,332.58,19.733,259.79,14.05,95518,91782
2,2012-01-09,330.30,-0.009372,2368.570,0.00,13.9753,24.12,0.5,3.1,3.3,...,28.782,1045.443,28.85,9369.4,331.04,19.733,259.79,14.05,69482,90598
3,2012-01-10,332.20,0.005736,2447.349,0.00,14.1068,23.68,0.5,3.1,3.3,...,29.815,1056.947,29.69,8523.4,332.05,19.733,259.79,14.05,63840,86898
4,2012-01-11,336.20,0.011969,2435.608,0.00,14.1205,24.11,0.5,3.1,3.3,...,29.890,1064.405,29.81,5768.8,335.50,19.733,259.79,14.05,61830,85756


In [16]:
# Data preprocessing

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df = df.dropna()  

# Define variable groups
group1_vars = ['HS300', 'CompriceIndex', 'PE', 'S&P500Volatility3', 'deprate', 'deprate3', 'deprate6', 'CPI', 'Inflatindex']
group2_vars = ['CHIUncertaintyIndex', 'CrudeoilETFVIX', 'USUncertaintyIndex', 'Geopoliticalriskindex', 'interest', 
               'dollarindex', 'setpriceCOMEX', 'clspriceCOMEX', 'tradevolumeCOMEX', 'positionCOMEX']

In [17]:
# Function to perform PCA and generate visualizations
def perform_pca(df, variables):
    X = df[variables]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    # Select principal components with eigenvalues > 1
    eigenvalues = pca.explained_variance_
    selected_components = [i for i, eigen in enumerate(eigenvalues) if eigen > 1]
    
    # Save results
    pca_results = pd.DataFrame({
        'Principal Component': [f'PC{i+1}' for i in range(len(eigenvalues))],
        'Eigenvalue': eigenvalues,
        'Explained Variance Ratio': pca.explained_variance_ratio_
    })
    X_selected = X_pca[:, selected_components]
    
    return X_selected, pca_results, selected_components

In [18]:
# Perform PCA for Group 1
group1_pca, group1_results, group1_selected = perform_pca(df, group1_vars)
group1_results.to_csv('group1_pca_results.csv', index=False)

# Perform PCA for Group 2
group2_pca, group2_results, group2_selected = perform_pca(df, group2_vars)
group2_results.to_csv('group2_pca_results.csv', index=False)

# Combine selected principal components
group1_selected_pca = pd.DataFrame(group1_pca, columns=[f'g1_PC{i+1}' for i in range(group1_pca.shape[1])])
group2_selected_pca = pd.DataFrame(group2_pca, columns=[f'g2_PC{i+1}' for i in range(group2_pca.shape[1])])

combined_pca = pd.concat([group1_selected_pca, group2_selected_pca], axis=1)


In [19]:

# Save the final principal component data
output_path = r"C:\Users\lenovo‘\Desktop\pcanewss.xlsx"
combined_pca.to_excel(output_path, index=False)
print(f"PCA save at: {output_path}")

主成分表已保存至: C:\Users\lenovo‘\Desktop\pcanewss.xlsx
